# Modelado tabular con Autgluon

In [1]:
!sudo apt-get update
!sudo apt-get install gcsfuse

Get:1 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease [1477 B]
Hit:2 https://deb.debian.org/debian bullseye InRelease                         
Hit:3 https://download.docker.com/linux/debian bullseye InRelease   
Get:4 https://deb.debian.org/debian-security bullseye-security InRelease [27.2 kB]
Get:5 https://deb.debian.org/debian bullseye-updates InRelease [44.0 kB]
Get:6 https://deb.debian.org/debian bullseye-backports InRelease [48.9 kB]
Hit:7 https://packages.cloud.google.com/apt gcsfuse-bullseye InRelease
Hit:8 https://packages.cloud.google.com/apt google-compute-engine-bullseye-stable InRelease
Hit:9 https://packages.cloud.google.com/apt cloud-sdk-bullseye InRelease
Get:10 https://deb.debian.org/debian-security bullseye-security/main Sources [247 kB]
Get:11 https://deb.debian.org/debian-security bullseye-security/main amd64 Packages [384 kB]
Get:12 https://deb.debian.org/debian-security bullseye-security/main Translation-en [255 kB]
Hit:13 https://packages.

In [2]:
#!pip install autogluon.timeseries

# Carga librerías

In [2]:
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor
from autogluon.tabular import TabularPredictor
import numpy as np

In [4]:
!mkdir -p /home/jupyter/franco_maestria/gcs_model_dir_fullpower_hibrido_top125_v3

In [3]:
#!fusermount -u /home/jupyter/franco_maestria/gcs_model_dir_fullpower_hibrido_top100_v2

In [5]:
!gcsfuse forecasting_customer_product /home/jupyter/franco_maestria/gcs_model_dir_fullpower_hibrido_top125_v3

{"timestamp":{"seconds":1752969862,"nanos":656759083},"severity":"INFO","message":"Start gcsfuse/3.1.0 (Go version go1.24.0) for app \"\" using mount point: /home/jupyter/franco_maestria/gcs_model_dir_fullpower_hibrido_top125_v3\n"}
{"timestamp":{"seconds":1752969862,"nanos":656812859},"severity":"INFO","message":"GCSFuse config","config":{"AppName":"","CacheDir":"","Debug":{"ExitOnInvariantViolation":false,"Fuse":false,"Gcs":false,"LogMutex":false},"DisableAutoconfig":false,"EnableAtomicRenameObject":true,"EnableGoogleLibAuth":false,"EnableHns":true,"EnableNewReader":false,"FileCache":{"CacheFileForRangeRead":false,"DownloadChunkSizeMb":200,"EnableCrc":false,"EnableODirect":false,"EnableParallelDownloads":false,"ExperimentalExcludeRegex":"","ExperimentalParallelDownloadsDefaultOn":true,"MaxParallelDownloads":96,"MaxSizeMb":-1,"ParallelDownloadsPerFile":16,"WriteBufferSize":4194304},"FileSystem":{"DirMode":"755","DisableParallelDirops":false,"ExperimentalEnableDentryCache":false,"Exper

In [6]:
# ------------------------
# 1) Cargar parquet con FE
# ------------------------

parquet_path = "panel_cliente_producto_fe.parquet"
df_modelo = pd.read_parquet(parquet_path)

# ------------------------
# 2) Cargar Prophet features
# ------------------------
csv_path = "prophet_features_customer_product.csv"
df_prophet = pd.read_csv(csv_path)
print(f"✅ CSV Prophet cargado. Shape: {df_prophet.shape}")

# ------------------------
# 3) Asegurar consistencia de tipos
# ------------------------
df_modelo['fecha'] = pd.to_datetime(df_modelo['fecha'])
df_prophet['fecha'] = pd.to_datetime(df_prophet['fecha'])

# ------------------------
# 4) Realizar join por 'product_id' y 'fecha'
# ------------------------
df_modelo_final = df_modelo.merge(
    df_prophet,
    on=['customer_id','product_id', 'fecha'],
    how='left'
)
print(f"✅ Merge completado. Shape final: {df_modelo_final.shape}")

# ------------------------
# 5) Convertir columnas float64 a float32 para ahorrar memoria
# ------------------------
float64_cols = df_modelo_final.select_dtypes(include=['float64']).columns.tolist()

df_modelo_final[float64_cols] = df_modelo_final[float64_cols].astype('float32')

print(f"✅ Conversión de float64 a float32 completada para columnas: {float64_cols}")
print(df_modelo_final.info())

print(f"✅ Merge completado. Shape final: {df_modelo_final.shape}")

# Verifica el resultado
df_modelo_final.head()


print(f"✅ Parquet cargado. Shape: {df_modelo_final.shape}")

✅ CSV Prophet cargado. Shape: (7249573, 9)
✅ Merge completado. Shape final: (12138186, 200)
✅ Conversión de float64 a float32 completada para columnas: ['tn_x', 'inflacion', 'cambio_dolar', 'stock_final', 'clase', 'tn_1', 'diff_tn_1', 'tn_2', 'diff_tn_2', 'tn_3', 'diff_tn_3', 'tn_4', 'diff_tn_4', 'tn_5', 'diff_tn_5', 'tn_6', 'diff_tn_6', 'tn_7', 'diff_tn_7', 'tn_8', 'diff_tn_8', 'tn_9', 'diff_tn_9', 'tn_10', 'diff_tn_10', 'tn_11', 'diff_tn_11', 'tn_12', 'diff_tn_12', 'tn_13', 'diff_tn_13', 'tn_14', 'diff_tn_14', 'tn_15', 'diff_tn_15', 'tn_16', 'diff_tn_16', 'tn_17', 'diff_tn_17', 'tn_18', 'diff_tn_18', 'tn_19', 'diff_tn_19', 'tn_20', 'diff_tn_20', 'tn_21', 'diff_tn_21', 'tn_22', 'diff_tn_22', 'tn_23', 'diff_tn_23', 'tn_24', 'diff_tn_24', 'tn_25', 'diff_tn_25', 'tn_26', 'diff_tn_26', 'tn_27', 'diff_tn_27', 'tn_28', 'diff_tn_28', 'tn_29', 'diff_tn_29', 'tn_30', 'diff_tn_30', 'tn_31', 'diff_tn_31', 'tn_32', 'diff_tn_32', 'tn_33', 'diff_tn_33', 'tn_34', 'diff_tn_34', 'tn_35', 'diff_tn_35',

In [7]:
# =============================================
# 📦 BLOQUE 1 — Filtrar TOP 70 clientes y resto
# =============================================

# 2) Calcular compra promedio por cliente
cliente_avg = (
    df_modelo_final.groupby('customer_id')['tn_x']
    .mean()
    .reset_index()
    .rename(columns={'tn_x': 'avg_tn'})
    .sort_values('avg_tn', ascending=False)
)

# 3) Identificar TOP 20 clientes
top_40_customers = cliente_avg.head(125)['customer_id'].tolist()
print(f"TOP 40 clientes:\n{top_40_customers}")

# 4) Crear datasets
df_top40 = df_modelo_final[df_modelo_final['customer_id'].isin(top_40_customers)].copy()
df_otros = df_modelo_final[~df_modelo_final['customer_id'].isin(top_40_customers)].copy()

print(f"TOP40 shape: {df_top40.shape} | Otros shape: {df_otros.shape}")


TOP 40 clientes:
[10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10010, 10011, 10012, 10013, 10014, 10016, 10015, 10017, 10018, 10019, 10020, 10023, 10022, 10021, 10024, 10025, 10039, 10026, 10028, 10030, 10033, 10027, 10034, 10032, 10031, 10038, 10035, 10037, 10036, 10042, 10041, 10082, 10029, 10045, 10044, 10043, 10040, 10047, 10046, 10074, 10051, 10048, 10050, 10057, 10049, 10053, 10055, 10054, 10127, 10052, 10060, 10056, 10064, 10061, 10062, 10059, 10063, 10058, 10066, 10065, 10067, 10069, 10072, 10071, 10068, 10076, 10073, 10070, 10075, 10081, 10080, 10084, 10086, 10088, 10077, 10078, 10090, 10089, 10079, 10091, 10083, 10085, 10097, 10096, 10087, 10094, 10092, 10093, 10101, 10099, 10098, 10095, 10102, 10104, 10105, 10109, 10113, 10103, 10107, 10108, 10110, 10112, 10106, 10136, 10111, 10115, 10118, 10117, 10121, 10100, 10114, 10116, 10130, 10119, 10122, 10125]
TOP40 shape: (2763524, 200) | Otros shape: (9374662, 200)


In [ ]:
# ====================================================
# 🚀 BLOQUE 2 — Modelo TABULAR para TOP 40 clientes
# ====================================================

# ⚙️ Separar Train/Test
df_top40['fecha'] = pd.to_datetime(df_top40['fecha'])
train_top40 = df_top40[(df_top40['fecha'] <= '2019-10-01') & df_top40['clase'].notnull()].copy()
test_top40 = df_top40[df_top40['fecha'] == '2019-12-01'].copy()

# Escalar magnitud de toneladas vendidas
train_top40['sample_weight'] = train_top40['tn_x']

features_top40 = [col for col in df_top40.columns if col not in ['periodo', 'clase', 'tn_y','seasonal']]

# ⚙️ Entrenar predictor
predictor_top40 = TabularPredictor(label='clase', problem_type='regression', eval_metric='mae',
    path='gcs_model_dir_fullpower_hibrido_top125_v3')
predictor_top40.fit(
    train_data=train_top40[features_top40 + ['clase']],
    presets='best_quality',
    time_limit=32400,
    ag_args_fit={'sample_weight': 'sample_weight'}
)

# ⚙️ Predicción y agregado por producto
test_top40['tn_pred'] = predictor_top40.predict(test_top40[features_top40])
df_top40_pred = (
    test_top40.groupby('product_id')['tn_pred']
    .sum()
    .reset_index()
    .rename(columns={'tn_pred': 'tn'})
)
print(df_top40_pred.head())

# ⚙️ Guardar CSV parcial
df_top40_pred.to_csv('forecast_top125_202002.csv', index=False)
print("✅ Forecast TOP100 guardado.")


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.237-1 (2025-05-19)
CPU Count:          48
Memory Avail:       269.68 GB / 377.89 GB (71.4%)
Disk Space Avail:   1048576.00 GB / 1048576.00 GB (100.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke

(_ray_fit pid=112502) [1000]	valid_set's l1: 0.280147
(_ray_fit pid=112501) [1000]	valid_set's l1: 0.280777 [repeated 5x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=112504) [1000]	valid_set's l1: 0.279575 [repeated 2x across cluster]
(_ray_fit pid=112499) [2000]	valid_set's l1: 0.284572
(_ray_fit pid=112502) [2000]	valid_set's l1: 0.276846
(_ray_fit pid=112497) [2000]	valid_set's l1: 0.279054 [repeated 2x across cluster]
(_ray_fit pid=112498) [2000]	valid_set's l1: 0.274577 [repeated 2x across cluster]
(_ray_fit pid=112501) [2000]	valid_set's l1: 0.277759 [repeated 2x across cluster]
(_ray_fit pid=112499) [3000]	valid_set's l1: 0.282753
(_ray_fit pid=112502) [3000]	valid_set's l1: 0.275253
(_ray_fit pid=112497) [3000]	valid_set's l1: 0.277393 [repeated 2x across cluster]
(_ray_fit pid=11250

(_dystack pid=108576) 	-0.2727	 = Validation score   (-mean_absolute_error)
(_dystack pid=108576) 	1684.74s	 = Training   runtime
(_dystack pid=108576) 	694.05s	 = Validation runtime
(_dystack pid=108576) Fitting model: LightGBM_BAG_L1 ... Training model for up to 3579.60s of the 6269.08s of remaining time.
(_dystack pid=108576) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=6, gpus=0, memory=3.93%)


(_ray_fit pid=124532) [1000]	valid_set's l1: 0.278667
(_ray_fit pid=124530) [1000]	valid_set's l1: 0.279921
(_ray_fit pid=124528) [1000]	valid_set's l1: 0.276359 [repeated 5x across cluster]
(_ray_fit pid=124532) [2000]	valid_set's l1: 0.275105 [repeated 2x across cluster]
(_ray_fit pid=124534) [2000]	valid_set's l1: 0.280762 [repeated 2x across cluster]
(_ray_fit pid=124529) [2000]	valid_set's l1: 0.281321 [repeated 4x across cluster]
(_ray_fit pid=124532) [3000]	valid_set's l1: 0.272708 [repeated 2x across cluster]
(_ray_fit pid=124534) [3000]	valid_set's l1: 0.278327 [repeated 2x across cluster]
(_ray_fit pid=124533) [3000]	valid_set's l1: 0.274165 [repeated 4x across cluster]
(_ray_fit pid=124532) [4000]	valid_set's l1: 0.271139 [repeated 2x across cluster]
(_ray_fit pid=124535) [4000]	valid_set's l1: 0.271207
(_ray_fit pid=124534) [4000]	valid_set's l1: 0.277002
(_ray_fit pid=124533) [4000]	valid_set's l1: 0.272699 [repeated 5x across cluster]
(_ray_fit pid=124532) [5000]	valid_se

(_dystack pid=108576) 	-0.2693	 = Validation score   (-mean_absolute_error)
(_dystack pid=108576) 	1719.32s	 = Training   runtime
(_dystack pid=108576) 	787.78s	 = Validation runtime
(_dystack pid=108576) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 1767.40s of the 4456.88s of remaining time.
(_dystack pid=108576) 	Warning: Exception caused RandomForestMSE_BAG_L1 to fail during training... Skipping this model.
(_dystack pid=108576) 		Input X contains infinity or a value too large for dtype('float32').
(_dystack pid=108576) Detailed Traceback:
(_dystack pid=108576) Traceback (most recent call last):
(_dystack pid=108576)   File "/opt/conda/lib/python3.10/site-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2169, in _train_and_save
(_dystack pid=108576)     model = self._train_single(**model_fit_kwargs)
(_dystack pid=108576)   File "/opt/conda/lib/python3.10/site-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2055, in _train_single
(_dys

...

(_dystack pid=108576) 	-0.2717	 = Validation score   (-mean_absolute_error)
(_dystack pid=108576) 	479.62s	 = Training   runtime
(_dystack pid=108576) 	50.26s	 = Validation runtime
(_dystack pid=108576) Fitting model: LightGBM_BAG_L2 ... Training model for up to 2109.40s of the 2108.46s of remaining time.
(_dystack pid=108576) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=6, gpus=0, memory=4.07%)


(_ray_fit pid=156963) [1000]	valid_set's l1: 0.266791
(_ray_fit pid=156960) [1000]	valid_set's l1: 0.262813 [repeated 2x across cluster]


(_dystack pid=108576) 	-0.2691	 = Validation score   (-mean_absolute_error)
(_dystack pid=108576) 	160.56s	 = Training   runtime
(_dystack pid=108576) 	7.71s	 = Validation runtime
(_dystack pid=108576) Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 1931.95s of the 1931.01s of remaining time.
(_dystack pid=108576) 	Warning: Exception caused RandomForestMSE_BAG_L2 to fail during training... Skipping this model.
(_dystack pid=108576) 		Input X contains infinity or a value too large for dtype('float32').
(_dystack pid=108576) Detailed Traceback:
(_dystack pid=108576) Traceback (most recent call last):
(_dystack pid=108576)   File "/opt/conda/lib/python3.10/site-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2169, in _train_and_save
(_dystack pid=108576)     model = self._train_single(**model_fit_kwargs)
(_dystack pid=108576)   File "/opt/conda/lib/python3.10/site-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2055, in _train_single
(_dystac

(_ray_fit pid=173638) [1000]	valid_set's l1: 0.263745


(_ray_fit pid=173637) 	Ran out of time, early stopping on iteration 918. Best iteration is:
(_ray_fit pid=173637) 	[788]	valid_set's l1: 0.273283
(_dystack pid=108576) 	-0.2688	 = Validation score   (-mean_absolute_error)
(_dystack pid=108576) 	184.11s	 = Training   runtime
(_dystack pid=108576) 	16.09s	 = Validation runtime
(_dystack pid=108576) Fitting model: CatBoost_r177_BAG_L2 ... Training model for up to 36.47s of the 35.52s of remaining time.
(_ray_fit pid=173638) 	Ran out of time, early stopping on iteration 1007. Best iteration is: [repeated 3x across cluster]
(_ray_fit pid=173638) 	[905]	valid_set's l1: 0.26372 [repeated 3x across cluster]
(_dystack pid=108576) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=6, gpus=0, memory=4.27%)
(_ray_fit pid=182709) 	Ran out of time, early stopping on iteration 1.
(_dystack pid=108576) 	-0.5957	 = Validation score   (-mean_absolute_error)
(_dystack pid=108576) 	15.32s	 = Trainin

In [12]:
# -------------------------------
# 9) Leaderboard (performance interna)
# -------------------------------
print("\n🔍 Leaderboard:")
lb = predictor_top40.leaderboard(silent=True)
print(lb)

# -------------------------------
# 10) Importancia de features
# -------------------------------
print("\n🔍 Importancia de Features:")
fi = predictor_top40.feature_importance(train_top40[features_top40 + ['clase']])
fi = fi.reset_index().rename(columns={'index': 'feature'})
print(fi.head(100))
fi.to_csv('importancia_feature_modelo_customer_product_hib_20-07-25_Autogluon.csv', index=False)


🔍 Leaderboard:
                   model  score_val          eval_metric  pred_time_val  \
0    WeightedEnsemble_L3  -0.260462  mean_absolute_error    2350.852081   
1    WeightedEnsemble_L2  -0.261105  mean_absolute_error    2246.332838   
2   LightGBMLarge_BAG_L1  -0.261951  mean_absolute_error     888.315150   
3   LightGBMLarge_BAG_L2  -0.264451  mean_absolute_error    2268.687229   
4        LightGBM_BAG_L2  -0.264679  mean_absolute_error    2263.143661   
5        CatBoost_BAG_L2  -0.264681  mean_absolute_error    2255.159857   
6   CatBoost_r177_BAG_L2  -0.265808  mean_absolute_error    2254.410132   
7   LightGBM_r131_BAG_L2  -0.265816  mean_absolute_error    2265.896403   
8      LightGBMXT_BAG_L2  -0.266684  mean_absolute_error    2315.362953   
9        LightGBM_BAG_L1  -0.267526  mean_absolute_error     628.072803   
10     LightGBMXT_BAG_L1  -0.271217  mean_absolute_error     721.900634   
11       CatBoost_BAG_L1  -0.272542  mean_absolute_error       8.017347   
12  CatBo

In [ ]:
# ============================================================
# 🔄 BLOQUE 3 — Modelo SERIES DE TIEMPO para resto de clientes
# ============================================================

from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

# ⚙️ 1) Agregar por producto
df_resto_prod = (
    df_otros.groupby(['product_id', 'fecha'], as_index=False)
    .agg({'tn_x': 'sum'})
)

# 👉 Renombrar para TimeSeriesDataFrame
df_resto_prod = df_resto_prod.rename(columns={
    'product_id': 'item_id',
    'fecha': 'timestamp'
})
df_resto_prod['timestamp'] = pd.to_datetime(df_resto_prod['timestamp'])
df_resto_prod = df_resto_prod.sort_values(['item_id', 'timestamp'])

# ⚙️ 2) Crear objeto TimeSeriesDataFrame
ts_df = TimeSeriesDataFrame.from_data_frame(
    df_resto_prod,
    id_column='item_id',
    timestamp_column='timestamp'
)

print(ts_df.head())

# ⚙️ 3) Configurar predictor
predictor_resto = TimeSeriesPredictor(
    target='tn_x',
    prediction_length=2,
    freq='M',
    eval_metric='MAE'
)

# ⚙️ 4) Entrenar
predictor_resto.fit(ts_df, num_val_windows=2, time_limit=14400)

# ⚙️ 5) Predecir
forecasts = predictor_resto.predict(ts_df)
forecasts_df = forecasts.reset_index().groupby('item_id')['mean'].sum().reset_index()
forecasts_df.columns = ['product_id', 'tn']

print(forecasts_df.head())

# ⚙️ 6) Guardar
forecasts_df.to_csv('forecast_resto_top125_202002.csv', index=False)
print("✅ Forecast resto clientes guardado.")



In [ ]:
# ================================================
# 🗃️ BLOQUE 4 — Merge forecasts y salida final
# ================================================

df_top40_pred = pd.read_csv('forecast_top125_202002.csv')
df_resto_pred = pd.read_csv('forecast_resto_top125_202002.csv')

# ⚙️ Unir y sumar por producto
df_final = (
    pd.concat([df_top40_pred, df_resto_pred], axis=0)
    .groupby('product_id', as_index=False)
    .agg({'tn': 'sum'})
)

print(df_final.head())

# ⚙️ Guardar archivo final
df_final.to_csv('forecast_total_top_125_202002.csv', index=False)
print("✅ Forecast combinado guardado: forecast_total_202002.csv")
print(f"Productos únicos: {df_final['product_id'].nunique()} | TN totales: {df_final['tn'].sum():,.2f}")
